In [1]:
from multiprocessing import Pool, Process
from datetime import datetime
import traceback
import requests
import os, re
import time
import json

In [2]:
IS_JUPYTER = True
ROOT_PATH  = '/home/jovyan/project' if IS_JUPYTER else '/config/workspace/project'

JSON_PATH        = f'{ROOT_PATH}/utils/keys'
JSON_BACKUP_PATH = f'{ROOT_PATH}/BACKUPS/keys'

In [3]:
load_json = lambda json_path: json.loads(open(json_path, 'r').read())
save_json = lambda data, json_path: json.dump(data, open(json_path, 'w'))

def repair_keys(json_path):
    try: keys = load_json(f'{json_path}/keys.json')
        
    except Exception as e:
        print(f'[ERR.K-002] json 파일이 깨져 열 수 없습니다. {e}')
        text = open(f'{JSON_BACKUP_PATH}/keys.txt', 'r').read().split('\n')
        keys = {platform : key 
                for platform, key in zip(['youtube', 'steam'], text)}
        
        save_paths = [JSON_PATH, JSON_BACKUP_PATH]
        
        print(save_paths)
        for save_path in save_paths:
            
            print(save_path)
            os.makedirs(save_path, exist_ok = True)
            save_json(keys, f'{save_path}/keys.json')
        
    finally: return keys

def return_or_print(response):
    
    if response.status_code == 200: return response.json()
    else: print(f'no response data with code : {response.status_code}')

    
get_api  = lambda url: return_or_print(requests.get(url))

def get_info(appid):
    
    response = requests.get(f'https://store.steampowered.com/api/appdetails?appids={appid}&l=korean')
    return return_or_print(response)

In [4]:
if os.path.isfile(f'{JSON_PATH}/keys.json'):
    key = repair_keys(JSON_PATH)['steam']
        
else:
    print(f'[ERR.K-001] json 파일이 존재하지 않아 백업 데이터를 로딩합니다.')
    key = repair_keys(JSON_BACKUP_PATH)['steam']

In [5]:
unix2datetime = lambda unixtime: str(datetime.fromtimestamp(unixtime))
def get_user_library(key, games):
    
    library_url = f'''https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={key}&steamid={user_id}&format=json'''
    games = get_api(library_url)['response']['games']
    
    user_datas = []
    for game in games:
        try:
            last_play_time = unix2datetime(game['rtime_last_played'])
            
            info = [game['appid'], game['playtime_forever'], str(last_play_time)]
            user_datas.append(info)
            
        except Exception as e:
            print(traceback.format_exc(), e)
            
    return user_datas


In [6]:

user_id = 76561198032052956
game_info  = get_user_library(key, user_id)


In [7]:
len(requests.get('https://api.steampowered.com/ISteamApps/GetAppList/v2/').json()['applist']['apps'])

152392

In [9]:
s = time.time()
json.loads(open(f'jsons/1181910/1181910.json', 'r').read())
time.time() - s

0.07257914543151855

In [ ]:
s = time.time()
get_info(1647670)
time.time() - s

In [ ]:
get_news = lambda appid: get_api(f'http://api.steampowered.com/ISteamNews/GetNewsForApp/v0002/?appid={appid}')


In [ ]:
news = get_news(1244370)
news

In [16]:
from xml.dom.minidom import parseString

In [15]:
response = requests.get('http://steamcommunity.com/id/KimHippo/games?tab=all&xml=1')

In [18]:
steamid = str(parseString(response.text.encode('utf-8')).getElementsByTagName('steamID64'))

In [24]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><error><![CDATA[The specified profile could not be found.]]></error></response>'